In [1]:
# Dependencies and Setup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib import rcParams
import scipy.stats as sts
import os
from collections import Counter
import requests
import json

# Google API Key
from api_keys import gkey

In [2]:
#setting background of figure black
plt.style.use('ggplot')

In [3]:
#Set Fonts/Plot Style
rcParams['figure.figsize'] = [15.0, 10.0]
rcParams['figure.dpi'] = 80
rcParams['savefig.dpi'] = 100
rcParams['font.size'] = 20
rcParams['legend.fontsize'] = 'large'
rcParams['figure.titlesize'] = 'medium'
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']
rcParams['xtick.direction'] = 'in'
rcParams['ytick.direction'] = 'in'
rcParams['xtick.top'] = True
rcParams['ytick.right'] = True
rcParams['xtick.major.size'] = 10
rcParams['ytick.major.size'] = 10
rcParams['xtick.minor.size'] = 5
rcParams['ytick.minor.size'] = 5
rcParams['xtick.minor.visible'] = True
rcParams['ytick.minor.visible'] = True

## Looking for 2012 US census data

In [ ]:
# 2012 census data for all US zipcodes
#reading 2012 combined census data
path_comb_2012=os.path.join('output_census', 'census_comb_2012.csv')
usa_2012=pd.read_csv(path_comb_2012)

In [ ]:
#looking for usa_2012 dataframe
usa_2012.head()

In [ ]:
#columns of 2012  dataframe
usa_2012.columns

In [ ]:
#shape of the dataframe
usa_2012.shape

In [ ]:
#removing the rows with house value less than 1
usa_2012=(usa_2012[(usa_2012['House Value']>0)&
                (usa_2012['Household Income']>0)&
                (usa_2012['Monthly Owner Cost']>0)])

In [ ]:
#looking for null value
usa_2012.isna().sum()

In [ ]:
#removing the rows with lat and lng missing value
usa_2012.dropna(subset=['Lat', 'Lng', 
                        'Public Transport Rate',
                        'Personal Transport Rate','Per Capita Income'], 
                         inplace=True)

In [ ]:
#shape of the clean dataframe
usa_2012.shape

In [ ]:
#info of the dataframe
usa_2012.info()

In [ ]:
#describing the dataframe
usa_2012.describe()

## Looking for 2012 California census data

In [ ]:
#reading 2012 combined california census data
path_ca_2012=os.path.join('output_census', 'census_ca_2012.csv')
ca_2012=pd.read_csv(path_ca_2012)

In [ ]:
#looking for ca_2012 dataframe
ca_2012.head()

In [ ]:
#columns of 2012  dataframe
ca_2012.columns

In [ ]:
#shape of the dataframe
ca_2012.shape

In [ ]:
#removing the rows with house value less than 1
#removing the rows with house value less than 1
ca_2012=(ca_2012[(ca_2012['House Value']>0)&
                (ca_2012['Household Income']>0)&
                (ca_2012['Monthly Owner Cost']>0)])

In [ ]:
#looking for null value
ca_2012.isna().sum()

In [ ]:
#removing the rows with lat and lng missing value
ca_2012.dropna(subset=['Public Transport Rate',
                        'Personal Transport Rate'], inplace=True)

In [ ]:
#shape of the clean dataframe
ca_2012.shape

In [ ]:
#converting minutes to hour

ca_2012['Commute Time Public']=ca_2012['Commute Time Public']/60
ca_2012['Commute Time Car']=ca_2012['Commute Time Car']/60

In [ ]:
#converting to int
ca_2012['Population']=ca_2012['Population'].astype(int)

In [ ]:
#info of the dataframe
ca_2012.info()

In [ ]:
#describing the dataframe
ca_2012.describe()

In [ ]:
#selecting only important columns
ca_12=ca_2012[['City','Population', 'House Value','Household Income',
       'Poverty Rate', 'Unemployment Rate','Monthly Owner Cost', 'Monthly Rent',
       'Public Transport Rate', 'Uneducated Rate' ]]

In [ ]:
#only selecting rows with +ve owner cost and rent
ca_12=(ca_12[(ca_12['Monthly Owner Cost']>0)&
             (ca_12['Monthly Rent']>0)])

In [ ]:
ca_12.shape

In [ ]:
#formating columns with 2 decimal place
ca_12[['House Value','Household Income',
       'Poverty Rate', 'Unemployment Rate','Monthly Owner Cost', 'Monthly Rent',
       'Public Transport Rate', 'Commute Time Public', 'Commute Time Car', 
       'Uneducated Rate' ]]=(ca_12[[ 'House Value','Household Income',
                                   'Poverty Rate', 'Unemployment Rate',
                                     'Monthly Owner Cost', 'Monthly Rent',
                                    'Public Transport Rate', 'Commute Time Public',
                                     'Commute Time Car', 'Uneducated Rate']]
                                                   .applymap('{:.2f}'.format))

In [ ]:
#using groupby method to groupby the data accordint to city
ca_12_city=ca_12.groupby('City').agg({'Population':'sum',
                              'House Value':'mean',
                               'Household Income':'mean',
                               'Poverty Rate':'mean',
                               'Unemployment Rate':'mean',
                               'Monthly Owner Cost':'mean',
                               'Monthly Rent':'mean',
                               'Public Transport Rate':'mean',
                               'Uneducated Rate':'mean'}).reset_index()

In [ ]:
#sorting data according to the population
ca_12_city=ca_12_city.sort_values(by=['House Value',
                                'Monthly Owner Cost','Household Income',
                                'Poverty Rate','Unemployment Rate',
                                'Public Transport Rate','Monthly Rent', 
                                'Uneducated Rate' ], ascending=False)

In [ ]:
#printing the new sorted datasets
ca_12_city.head()

In [ ]:
#city with higher house value
ca_12_city.nlargest(10, 'House Value')

In [ ]:
#City with smallest poverty rate
ca_12_city.nsmallest(10, 'Poverty Rate')

In [ ]:
#city with smallest unemployment rate
ca_12_city.nsmallest(10, 'Unemployment Rate')

In [ ]:
#city with larger commute time
#ca_12_city.nsmallest(10, 'Commute Time Car')

In [ ]:
#city with smallest monthly owner cost
ca_12_city.nsmallest(10, 'Monthly Owner Cost')

In [ ]:
#city with largest household income
ca_12_city.nlargest(10, 'Household Income')

## Plotting 2012 california data

In [ ]:
#plotting all correlation using seaborn heatmap
corr_12=ca_12.corr()
import seaborn as sns
fig, ax=plt.subplots(figsize=(15,15))
sns.heatmap(corr_12,vmin=-1, vmax=1, ax=ax, cmap='BrBG')#annot=True,
plt.show()

In [ ]:
#plotting scatter plot of lat and lng
fig, ax=plt.subplots(figsize=(15,15))
ca_2012.plot(kind='scatter', x='Lng', y='Lat', alpha=0.4, 
                s=ca_2012['Population']/100,label='Population',
                 c='House Value', cmap=plt.get_cmap('rainbow'),
                colorbar=True, ax=ax)

plt.show()

In [ ]:
#plotting scatter plot of lat and lng
fig, ax=plt.subplots(figsize=(15,15))
ca_2012.plot(kind='scatter', x='Lng', y='Lat', alpha=0.3, 
                s=ca_2012['Population']/100,label='Population',
                 c='Household Income', cmap=plt.get_cmap('hsv'),
                colorbar=True, ax=ax)

plt.show()

## Looking for 2014 US census data

In [ ]:
# 2014 census data for all US zipcodes
#reading 2014 combined census data
path_comb_2014=os.path.join('output_census', 'census_comb_2014.csv')
usa_2014=pd.read_csv(path_comb_2014)

In [ ]:
#looking for usa_2014 dataframe
usa_2014.head()

In [ ]:
#columns of 2014  dataframe
usa_2014.columns

In [ ]:
#shape of the dataframe
usa_2014.shape

In [ ]:
#removing the rows with house value less than 1
usa_2014=usa_2014[usa_2014['House Value']>0]

In [ ]:
#looking for null value
usa_2014.isna().sum()

In [ ]:
#removing the rows with lat and lng missing value
usa_2014.dropna(subset=['Lat', 'Lng', 
                        'Public Transport Rate',
                        'Personal Transport Rate',
                        'Housing_units','County'], inplace=True)

In [ ]:
#shape of the clean dataframe
usa_2014.shape

In [ ]:
#info of the dataframe
usa_2014.info()

In [ ]:
#describing the dataframe
usa_2014.describe()

## Looking for 2014 California census data

In [ ]:
#reading 2014 combined california census data
path_ca_2014=os.path.join('output_census', 'census_ca_2014.csv')
ca_2014=pd.read_csv(path_ca_2014)

In [ ]:
#looking for ca_2014 dataframe
ca_2014.head()

In [ ]:
#columns of 2014  dataframe
ca_2014.columns

In [ ]:
#shape of the dataframe
ca_2014.shape

In [ ]:
#removing the rows with house value less than 1
ca_2014=ca_2014[ca_2014['House Value']>0]

In [ ]:
#looking for null value
ca_2014.isna().sum()

In [ ]:
#shape of the clean dataframe
ca_2014.shape

In [ ]:
#info of the dataframe
ca_2014.info()

In [ ]:
#describing the dataframe
ca_2014.describe()

## Looking for 2015 US census data

In [ ]:
# 2015 census data for all US zipcodes
#reading 2015 combined census data
path_comb_2015=os.path.join('output_census', 'census_comb_2015.csv')
usa_2015=pd.read_csv(path_comb_2015)

In [ ]:
#looking for usa_2015 dataframe
usa_2015.head()

In [ ]:
#columns of 2015  dataframe
usa_2015.columns

In [ ]:
#shape of the dataframe
usa_2015.shape

In [ ]:
#removing the rows with house value less than 1
usa_2015=usa_2015[usa_2015['House Value']>0]

In [ ]:
#looking for null value
usa_2015.isna().sum()

In [ ]:
#removing the rows with lat and lng missing value
usa_2015.dropna(subset=['Lat', 'Lng', 'Per Capita Income','County',
                        'Monthly Owner Cost','Monthly Rent',
                        'Public Transport Rate','Household Income',
                        'Personal Transport Rate'], 
                         inplace=True)

In [ ]:
#shape of the clean dataframe
usa_2015.shape

In [ ]:
#info of the dataframe
usa_2015.info()

In [ ]:
#describing the dataframe
usa_2015.describe()

## Looking for 2015 California census data

In [ ]:
#reading 2015 combined california census data
path_ca_2015=os.path.join('output_census', 'census_ca_2015.csv')
ca_2015=pd.read_csv(path_ca_2015)

In [ ]:
#looking for ca_2015 dataframe
ca_2015.head()

In [ ]:
#columns of 2015  dataframe
ca_2015.columns

In [ ]:
#shape of the dataframe
ca_2015.shape

In [ ]:
#removing the rows with house value less than 1
ca_2015=ca_2015[ca_2015['House Value']>0]

In [ ]:
#looking for null value
ca_2015.isna().sum()

In [ ]:
#removing the rows with lat and lng missing value
ca_2015.dropna(subset=['Household Income', 'Monthly Rent', 
                        'Monthly Owner Cost','Personal Transport Rate',
                      'Public Transport Rate'],inplace=True)

In [ ]:
#shape of the clean dataframe
ca_2015.shape

In [ ]:
#info of the dataframe
ca_2015.info()

In [ ]:
#describing the dataframe
ca_2015.describe()

## Looking for 2017 US census data

In [ ]:
# 2017 census data for all US zipcodes
#reading 2017 combined census data
path_comb_2017=os.path.join('output_census', 'census_comb_2017.csv')
usa_2017=pd.read_csv(path_comb_2017)

In [ ]:
#looking for usa_2017 dataframe
usa_2017.head()

In [ ]:
#columns of 20147  dataframe
usa_2017.columns

In [ ]:
#shape of the dataframe
usa_2017.shape

In [ ]:
#removing the rows with house value less than 1
usa_2017=usa_2017[usa_2017['House Value']>0]

In [ ]:
#looking for null value
usa_2017.isna().sum()

In [ ]:
#removing the rows with lat and lng missing value
usa_2017.dropna(subset=['Lat', 'Lng', 'Housing_units','County',
                        'Public Transport Rate','Personal Transport Rate'], 
                         inplace=True)

In [ ]:
#shape of the clean dataframe
usa_2017.shape

In [ ]:
#info of the dataframe
usa_2017.info()

In [ ]:
#describing the dataframe
usa_2017.describe()

## Looking for 2017 California census data

In [ ]:
#reading 2017 combined california census data
path_ca_2017=os.path.join('output_census', 'census_ca_2017.csv')
ca_2017=pd.read_csv(path_ca_2017)

In [ ]:
#looking for usa_2017 dataframe
ca_2017.head()

In [ ]:
#columns of 2017  dataframe
ca_2017.columns

In [ ]:
#shape of the dataframe
ca_2017.shape

In [ ]:
#removing the rows with house value less than 1
ca_2017=ca_2017[ca_2017['House Value']>0]

In [ ]:
#looking for null value
ca_2017.isna().sum()

In [ ]:
#removing the rows with lat and lng missing value


In [ ]:
#shape of the clean dataframe
ca_2017.shape

In [ ]:
#info of the dataframe
ca_2017.info()

In [ ]:
#describing the dataframe
ca_2017.describe()

## Looking for 2019 US census data

In [4]:
# 2019 census data for all US zipcodes
#reading 2019 combined census data
path_comb_2019=os.path.join('output_census', 'census_comb_2019.csv')
usa_2019=pd.read_csv(path_comb_2019)

In [5]:
#looking for usa_2014 dataframe
usa_2019.head()

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Rate,Unemployment Rate,House Value,House Construction Year,Monthly Owner Cost,...,White Population Rate,Black Population Rate,Hispanic Population Rate,Asian Population Rate,City,County,Lat,Lng,Housing_units,State
0,601,17113.0,41.9,14361.0,7493.0,61.660726,11.500029,83900.0,1981,771.0,...,73.721732,1.063519,99.643546,0.011687,Adjuntas,Adjuntas Municipio,NaN,NaN,NaN,PR
1,602,37751.0,42.9,16807.0,9694.0,49.410612,3.761490,85300.0,1978,877.0,...,85.240126,2.405234,92.469074,0.000000,Aguada,Aguada Municipio,18.36,-67.18,18073.0,PR
2,603,47081.0,42.1,16049.0,11259.0,50.319662,6.790425,118400.0,1980,832.0,...,79.350481,3.215735,97.368365,0.229392,Aguadilla,Aguadilla Municipio,18.45,-67.11,25653.0,PR
3,606,6392.0,44.3,12119.0,6093.0,65.472466,2.643930,80800.0,1978,526.0,...,45.525657,1.439299,99.155194,0.000000,Maricao,Maricao Municipio,18.20,-66.90,2877.0,PR
4,610,26686.0,42.7,19898.0,10572.0,45.731844,3.855954,87600.0,1978,751.0,...,82.597617,1.727498,95.061081,0.000000,Anasco,Aasco Municipio,18.28,-67.13,12618.0,PR


In [6]:
#columns of 2014  dataframe
usa_2019.columns

Index(['Zipcode', 'Population', 'Median Age', 'Household Income',
       'Per Capita Income', 'Poverty Rate', 'Unemployment Rate', 'House Value',
       'House Construction Year', 'Monthly Owner Cost', 'Monthly Rent',
       'Public Transport Rate', 'Personal Transport Rate',
       'Commute Time Public', 'Commute Time Car', 'High School Rate',
       'College Rate', 'Uneducated Rate', 'English Language',
       'Spanish Language', 'White Population Rate', 'Black Population Rate',
       'Hispanic Population Rate', 'Asian Population Rate', 'City', 'County',
       'Lat', 'Lng', 'Housing_units', 'State'],
      dtype='object')

In [7]:
#shape of the dataframe
usa_2019.shape

(33120, 30)

In [8]:
#removing the rows with house value less than 1
usa_2019=(usa_2019[(usa_2019['House Value']>0)&
                (usa_2019['Household Income']>0)&
                (usa_2019['Monthly Owner Cost']>0)])

In [9]:
#looking for null value
usa_2019.isna().sum()

Zipcode                         0
Population                      0
Median Age                      0
Household Income                0
Per Capita Income               0
Poverty Rate                    0
Unemployment Rate               0
House Value                     0
House Construction Year         0
Monthly Owner Cost              0
Monthly Rent                    0
Public Transport Rate           1
Personal Transport Rate         1
Commute Time Public         20708
Commute Time Car            20708
High School Rate                0
College Rate                    0
Uneducated Rate                 0
English Language            28941
Spanish Language            28941
White Population Rate           0
Black Population Rate           0
Hispanic Population Rate        0
Asian Population Rate           0
City                            0
County                          1
Lat                            16
Lng                            16
Housing_units                  16
State         

In [10]:
#removing the rows with lat and lng missing value
usa_2019.dropna(subset=['Lat', 'Lng','Housing_units','County', 
                        'Public Transport Rate','Personal Transport Rate'], 
                         inplace=True)

In [11]:
#shape of the clean dataframe
usa_2019.shape

(28923, 30)

In [12]:
#info of the dataframe
usa_2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28923 entries, 1 to 33119
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Zipcode                   28923 non-null  int64  
 1   Population                28923 non-null  float64
 2   Median Age                28923 non-null  float64
 3   Household Income          28923 non-null  float64
 4   Per Capita Income         28923 non-null  float64
 5   Poverty Rate              28923 non-null  float64
 6   Unemployment Rate         28923 non-null  float64
 7   House Value               28923 non-null  float64
 8   House Construction Year   28923 non-null  int64  
 9   Monthly Owner Cost        28923 non-null  float64
 10  Monthly Rent              28923 non-null  float64
 11  Public Transport Rate     28923 non-null  float64
 12  Personal Transport Rate   28923 non-null  float64
 13  Commute Time Public       8228 non-null   float64
 14  Commut

In [13]:
#describing the dataframe
usa_2019.describe()

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Rate,Unemployment Rate,House Value,House Construction Year,Monthly Owner Cost,...,Uneducated Rate,English Language,Spanish Language,White Population Rate,Black Population Rate,Hispanic Population Rate,Asian Population Rate,Lat,Lng,Housing_units
count,28923.000000,28923.000000,28923.000000,28923.000000,28923.000000,28923.000000,28923.000000,2.892300e+04,2.892300e+04,28923.000000,...,28923.000000,0.0,0.0,28923.000000,28923.000000,28923.000000,28923.000000,28923.000000,28923.000000,28923.000000
mean,49121.160184,11250.464336,42.459513,61886.646302,31680.641116,13.042679,2.457554,2.061185e+05,-1.594677e+05,1428.424022,...,0.721531,NaN,NaN,83.124381,7.852341,9.805983,2.343961,38.866237,-90.628141,4575.325554
std,27357.506521,15247.867882,7.784226,25958.786527,13621.932220,8.928869,1.830560,1.873462e+05,1.037032e+07,612.976365,...,0.943708,NaN,NaN,20.379255,15.557849,16.466417,5.679577,5.264032,14.817801,5762.985962
min,602.000000,24.000000,14.500000,6493.000000,3054.000000,0.000000,0.000000,9.999000e+03,-6.666667e+08,99.000000,...,0.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,17.960000,-173.000000,0.000000
25%,27029.000000,1154.000000,37.300000,45470.500000,23686.500000,6.620662,1.354366,1.016000e+05,1.964000e+03,1030.000000,...,0.097671,NaN,NaN,76.915201,0.000000,1.114428,0.000000,35.400000,-96.740000,596.000000
50%,49339.000000,3961.000000,41.900000,56268.000000,28748.000000,11.154656,2.187713,1.517000e+05,1.976000e+03,1235.000000,...,0.463939,NaN,NaN,91.508983,1.288530,3.629829,0.465230,39.680000,-88.000000,1824.000000
75%,71017.000000,16251.000000,46.900000,71543.000000,35665.500000,17.339537,3.161629,2.380000e+05,1.984000e+03,1621.000000,...,0.976494,NaN,NaN,96.848344,7.140009,10.089356,2.027271,42.200000,-80.130000,6797.500000
max,99929.000000,128294.000000,91.500000,250001.000000,187336.000000,85.142857,30.115830,2.000001e+06,2.014000e+03,4001.000000,...,26.595745,NaN,NaN,100.000000,100.000000,100.000000,75.288276,71.000000,-65.280000,47617.000000


In [14]:
usa_2019.columns

Index(['Zipcode', 'Population', 'Median Age', 'Household Income',
       'Per Capita Income', 'Poverty Rate', 'Unemployment Rate', 'House Value',
       'House Construction Year', 'Monthly Owner Cost', 'Monthly Rent',
       'Public Transport Rate', 'Personal Transport Rate',
       'Commute Time Public', 'Commute Time Car', 'High School Rate',
       'College Rate', 'Uneducated Rate', 'English Language',
       'Spanish Language', 'White Population Rate', 'Black Population Rate',
       'Hispanic Population Rate', 'Asian Population Rate', 'City', 'County',
       'Lat', 'Lng', 'Housing_units', 'State'],
      dtype='object')

In [16]:
usa_2019.drop(['Commute Time Car','Commute Time Public'], axis=1, inplace=True)

In [17]:
usa_2019.shape

(28923, 28)

In [18]:
#selecting only required columns
usa_2019_redu=usa_2019[['Population', 'Median Age', 'Household Income',
       'Per Capita Income', 'Poverty Rate', 'Unemployment Rate', 'House Value',
       'House Construction Year', 'Monthly Owner Cost', 'Monthly Rent',
       'Public Transport Rate', 'Personal Transport Rate',
        'High School Rate',
       'College Rate', 'Uneducated Rate', 'White Population Rate', 'Black Population Rate',
       'Hispanic Population Rate', 'Asian Population Rate', 'City', 
       'Lat', 'Lng', 'Housing_units']]

In [19]:
#saving csv file for machine learning
usa_2019_redu.to_csv('output_census/usa_2019_ml.csv', index=False)

## Looking for 2019 California census data

In [ ]:
#reading 2019 combined california census data
path_ca_2019=os.path.join('output_census', 'census_ca_2019.csv')
ca_2019=pd.read_csv(path_ca_2019)

In [ ]:
#looking for usa_2019 dataframe
ca_2019.head()

In [ ]:
#columns of 2019  dataframe
ca_2019.columns

In [ ]:
#shape of the dataframe
ca_2019.shape

In [ ]:
#removing the rows with house value less than 1
ca_2019=(ca_2019[(ca_2019['House Value']>0)&
                (ca_2019['Household Income']>0)&
                (ca_2019['Monthly Owner Cost']>0)])

In [ ]:
#looking for null value
ca_2019.isna().sum()

In [ ]:
#removing the rows with lat and lng missing value


In [ ]:
#shape of the clean dataframe
ca_2019.shape

In [ ]:
#info of the dataframe
ca_2019.info()

In [ ]:
#describing the dataframe
ca_2019.describe()

In [ ]:
#selecting only important columns
ca_19=ca_2019[['City','Population', 'House Value','Household Income',
       'Poverty Rate', 'Unemployment Rate','Monthly Owner Cost', 'Monthly Rent',
       'Public Transport Rate', 'Personal Transport Rate','College Rate','White Population Rate',
       'Uneducated Rate' ]]

In [ ]:
#only selecting rows with +ve owner cost and rent
ca_19=(ca_19[(ca_19['Monthly Owner Cost']>0)&
             (ca_19['Monthly Rent']>0)])

In [ ]:
#saving csv file for machine learning
ca_19.to_csv('output_census/ca_2019_ml.csv', index=False)

In [ ]:
#formating columns with 2 decimal place
ca_19[['House Value','Household Income',
       'Poverty Rate', 'Unemployment Rate','Monthly Owner Cost', 'Monthly Rent',
       'Public Transport Rate', 'Commute Time Public', 'Commute Time Car', 
       'Uneducated Rate' ]]=(ca_19[[ 'House Value','Household Income',
                                   'Poverty Rate', 'Unemployment Rate',
                                     'Monthly Owner Cost', 'Monthly Rent',
                                    'Public Transport Rate', 'Commute Time Public',
                                     'Commute Time Car', 'Uneducated Rate']]
                                                   .applymap('{:.2f}'.format))

In [ ]:
#using groupby method to groupby the data accordint to city
ca_19_city=ca_19.groupby('City').agg({'Population':'sum',
                              'House Value':'mean',
                               'Household Income':'mean',
                               'Poverty Rate':'mean',
                               'Unemployment Rate':'mean',
                               'Monthly Owner Cost':'mean',
                               'Monthly Rent':'mean',
                               'Public Transport Rate':'mean',
                               'Personal Transport Rate':'mean',
                                'College Rate':'mean',
                                'White Population Rate':'mean',      
                               'Uneducated Rate':'mean'}).reset_index()

In [ ]:
#sorting data according to the population
ca_19_city=ca_19_city.sort_values(by=['House Value',
                                'Monthly Owner Cost','Household Income',
                                'Poverty Rate','Unemployment Rate',
                                'Public Transport Rate','Monthly Rent', 
                                'Uneducated Rate' ], ascending=False)

In [ ]:
ca_19_city.shape

In [ ]:
#printing the new sorted datasets
ca_19_city.head()

In [ ]:
#city with higher house value
ca_19_city.nlargest(10, 'House Value')

In [ ]:
#City with smallest poverty rate
ca_19_city.nsmallest(10, 'Poverty Rate')

In [ ]:
#city with smallest unemployment rate
ca_19_city.nsmallest(10, 'Unemployment Rate')

In [ ]:
#city with larger commute time
ca_19_city.nsmallest(10, 'Commute Time Car')

In [ ]:
#city with smallest monthly owner cost
ca_19_city.nsmallest(10, 'Monthly Owner Cost')

In [ ]:
#city with largest household income
ca_19_city.nlargest(10, 'Household Income')

## Ploting 2019 California Data

In [ ]:
#plotting all correlation using seaborn heatmap
corr_19=ca_19.corr()
import seaborn as sns
fig, ax=plt.subplots(figsize=(15,15))
sns.heatmap(corr_19,vmin=-1, vmax=1, ax=ax, cmap='BrBG')#annot=True,
plt.show()

In [ ]:
corr_19=ca_19.corr()
corr_19.style.background_gradient(cmap='coolwarm')

In [ ]:
c1 = corr_19.abs().unstack()
c1.sort_values(ascending = False)[12:40]

In [ ]:
#plotting
fig, ax=plt.subplots(figsize=(15,15))
ca_2019.plot(kind='scatter', x='Lng', y='Lat', alpha=0.2, 
                s=ca_2019['Population']/100,label='Population',
                 c='House Value', cmap=plt.get_cmap('jet'),
                colorbar=True, ax=ax)

plt.show()